# 1505038

In [108]:
%pip install pandas

  Using cached pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [211]:
import numpy as np
import sys
import pandas as pd

### Global Variables

In [243]:
featureNo = 0 
classNo = 0 
sampleNo  = 0 
dataset = []
classLabels = []  



seed_val = 101
np.random.seed(seed_val)

### Read Train Dataset

In [244]:
def readTrainDataset(file_name): 
    global featureNo, classNo, sampleNo 
    global dataset, classLabels  

    with open(file_name, 'r', encoding='utf-8') as f: 
        lines = f.readlines()

    # first line contains 3 integers d m n. d = no. of features, m = no. of classes, n=no. of samples
    firstLine = [int(String) for String in lines[0].strip().split()]
    featureNo, classNo, sampleNo = firstLine[0], firstLine[1], firstLine[2]
    
    print(featureNo, classNo, sampleNo)

    for line in lines[1:]: 
        featureValuesForCurrentSample = list(map(float, line.strip().split()[:featureNo]))
        classOfCurrentSample = int(line.strip().split()[featureNo])
        # print(featureValuesForCurrentSample)
        dataset.append(np.array(featureValuesForCurrentSample, dtype=float))
        classLabels.append(classOfCurrentSample)
    print(featureNo)
    # print(classLabels)
    




In [245]:
readTrainDataset('dataset/trainLinearlySeparable.txt')
print(featureNo)

4 2 400
4
4


### Read Test Dataset

In [246]:
def readTestDataset(file_name): 
    global featureNo, classNo, sampleNo 

    testDataSet = []
    testClassLabels = []
    
    with open(file_name, 'r', encoding='utf-8') as f: 
        lines = f.readlines()

    # first line contains 3 integers d m n. d = no. of features, m = no. of classes, n=no. of samples

    for line in lines: 
        featureValuesForCurrentSample = list(map(float, line.strip().split()[:featureNo]))
        classOfCurrentSample = int(line.strip().split()[featureNo])
        # print(featureValuesForCurrentSample)

        testDataSet.append(np.array(featureValuesForCurrentSample, dtype=float))
        testClassLabels.append(classOfCurrentSample)

    return testDataSet, testClassLabels



## Basic Perceptron

In [253]:
class basicPerceptron: 
    def __init__(self): 
        self.w = np.random.uniform(-1, 1, featureNo + 1)
        self.learning_rate = 0.1

    def train(self): 
        global featureNo, classNo, sampleNo 
        global dataset, classLabels
        
        t = 0 
        T = 1000
        while t < T: 
            Y = [] # misclassified
            # print(featureNo, classNo, sampleNo)
            # print(dataset, classLabels)
            for i in range(sampleNo): 
                # print(len(dataset))
                x = np.array(dataset[i])
                # x = np.append(dataset[i], 1)            
                x = np.append(x, 1)            
                dotProduct = np.dot(self.w, x)
                if dotProduct < 0 and classLabels[i] == 1: 
                    delta = -1.0 
                    Y.append(x*delta)
                elif dotProduct >= 0 and classLabels[i] == 2: 
                    delta = 1.0  
                    Y.append(x*delta)
            
            if len(Y) == 0: 
                # training done! 100% accuracy. 
                print(f'Training successfully finished at {i}th iteration.')
                print("w: " + str(self.w) + "\n")
                break
           
            self.w = self.w - self.learning_rate * sum(Y)
            t = t + 1
        print(f'Exiting training after {T} steps.')


    def test(self,fileName): 
        log = {
                'sample_no' : [], 
                'feature_values' : [],
                'actual_class' : [],
                'predicted_class' : []
               }

        testDataSet, testClassLables = readTestDataset(fileName)
        # print(testDataSet)
        # print('##########')
        # print(testClassLabels)

        correctlyClassified = 0 

        for i in range(len(testDataSet)):
            data =  np.append(testDataSet[i], 1)
            # print(self.w)
            # print(data)
            dotProduct = np.dot(self.w, data) 
            if dotProduct > 0: 
                predictedClass = 1 
            else: 
                predictedClass = 2

            if predictedClass == testClassLables[i]: 
                correctlyClassified += 1
            else : 
                log['sample_no'].append(str(i))
                log['feature_values'].append(testDataSet[i])
                log['actual_class'].append(testClassLables[i])
                log['predicted_class'].append(predictedClass)
        
        log_frame = pd.DataFrame.from_dict(log) 
        accuracy = correctlyClassified/ len(testDataSet) * 100
        print(log_frame)
        print(f'- Accuracy {accuracy}%')




In [258]:
test_file_1 = "./dataset/testLinearlySeparable.txt"
train_file_1 = "./dataset/trainLinearlySeparable.txt"

readTrainDataset(train_file_1)
p = basicPerceptron()
p.train()
p.test(test_file_1)


4 2 400
4
Training successfully finished at 399th iteration.
w: [ 165.98066897    3.17838407   -6.80634708 -187.96197346 1758.44493875]

Exiting training after 1000 steps.
Empty DataFrame
Columns: [sample_no, feature_values, actual_class, predicted_class]
Index: []
- Accuracy 100.0%


## Reward and Punishment

In [261]:
class RewardAndPunishment: 
    def __init__(self): 
        self.w = np.random.uniform(-1, 1, featureNo + 1)
        self.learning_rate = 0.1

    def train(self): 
        global featureNo, classNo, sampleNo 
        global dataset, classLabels  
        
        t = 0 
        T = 1000
        while t < T: 
            Y = [] # misclassified
            flag = True
            for i in range(sampleNo): 
                # print(len(dataset))
                x = np.append(dataset[i], 1)   

                dotProduct = np.dot(self.w, x)
                if dotProduct < 0 and classLabels[i] == 1: 
                    self.w = self.w + self.learning_rate * x 
                    falg = False
                elif dotProduct >= 0 and classLabels[i] == 2: 
                    self.w = self.w - self.learning_rate * x
                    flag = False
            
            if flag: 
                print(f'Training stopping at {t} steps')
                print("weight vector", self.w)
                break

            t = t + 1
        print(f'Exiting training after {T} steps.')


    def test(self,fileName): 
        log = {
                'sample_no' : [], 
                'feature_values' : [],
                'actual_class' : [],
                'predicted_class' : []
               }

        testDataSet, testClassLables = readTestDataset(fileName)

        correctlyClassified = 0 

        for i in range(len(testDataSet)):
            data =  np.append(testDataSet[i], 1)
            # print(self.w)
            # print(data)
            dotProduct = np.dot(self.w, data) 
            if dotProduct > 0: 
                predictedClass = 1 
            else: 
                predictedClass = 2

            if predictedClass == testClassLables[i]: 
                correctlyClassified += 1
            else : 
                log['sample_no'].append(str(i))
                log['feature_values'].append(testDataSet[i])
                log['actual_class'].append(testClassLables[i])
                log['predicted_class'].append(predictedClass)
        
        log_frame = pd.DataFrame.from_dict(log) 
        accuracy = correctlyClassified/ len(testDataSet) * 100
        print(log_frame)
        print(f'- Accuracy {accuracy}%')




In [262]:
test_file_1 = "./dataset/testLinearlySeparable.txt"
train_file_1 = "./dataset/trainLinearlySeparable.txt"

readTrainDataset(train_file_1)
p = RewardAndPunishment()
p.train()
p.test(test_file_1)


4 2 400
4
Training stopping at 2 steps
weight vector [ 0.65568717  0.63260222 -0.13505908 -1.07536272  8.67527044]
Exiting training after 1000 steps.
Empty DataFrame
Columns: [sample_no, feature_values, actual_class, predicted_class]
Index: []
- Accuracy 100.0%


## Pocket Perceptron

In [263]:
class pocketPerceptron: 
    def __init__(self): 
        self.w = np.random.uniform(-1, 1, featureNo + 1)
        self.ws = self.w
        self.learning_rate = 0.1
        self.hs = 0 

    def train(self): 
        global featureNo, classNo, sampleNo 
        global dataset, classLabels
        
        t = 0 
        T = 1000
        while t < T: 
            Y = [] # misclassified
            for i in range(sampleNo): 
                x = np.append(dataset[i], 1)            
                dotProduct = np.dot(self.w, x)
                if dotProduct < 0 and classLabels[i] == 1: 
                    delta = -1.0 
                    Y.append(x*delta)
                elif dotProduct >= 0 and classLabels[i] == 2: 
                    delta = 1.0  
                    Y.append(x*delta)
            # print(len(dataset))
            if self.hs < len(dataset) - len(Y): # better found 
                self.hs = len(dataset) - len(Y)
                self.ws = self.w

            if len(Y) == 0: 
                # training done! 100% accuracy. 
                print(f'Training successfull finished at {i}th iteration.')
                print("ws: " + str(self.ws) + "\n")
                break
            else: 
                self.w = self.w - self.learning_rate * sum(Y)
            t = t + 1
        print(f'Exiting training after {T} steps.')


    def test(self,fileName): 
        log = {
                'sample_no' : [], 
                'feature_values' : [],
                'actual_class' : [],
                'predicted_class' : []
               }

        testDataSet, testClassLables = readTestDataset(fileName)

        correctlyClassified = 0 

        for i in range(len(testDataSet)):
            data =  np.append(testDataSet[i], 1)
            # print(self.w)
            # print(data)
            dotProduct = np.dot(self.ws, data) 
            if dotProduct > 0: 
                predictedClass = 1 
            else: 
                predictedClass = 2

            if predictedClass == testClassLables[i]: 
                correctlyClassified += 1
            else : 
                log['sample_no'].append(str(i))
                log['feature_values'].append(testDataSet[i])
                log['actual_class'].append(testClassLables[i])
                log['predicted_class'].append(predictedClass)
        
        log_frame = pd.DataFrame.from_dict(log) 
        accuracy = correctlyClassified/ len(testDataSet) * 100
        print(log_frame)
        print(f'- Accuracy {accuracy}%')




In [264]:
test_file_1 = "./dataset/testLinearlySeparable.txt"
train_file_1 = "./dataset/trainLinearlySeparable.txt"

test_file_2 = "./dataset/testLinearlyNonSeparable.txt"
train_file_2 = "./dataset/trainLinearlyNonSeparable.txt"

readTrainDataset(train_file_2)
p = pocketPerceptron()
p.train()
p.test(test_file_2)

4 2 400
4
Training successfull finished at 399th iteration.
ws: [ 44.80322648   2.19910692  -1.57578957 -48.11779648 432.15041017]

Exiting training after 1000 steps.
  sample_no                                     feature_values  actual_class  \
0       170  [1.608665812, 3.864370259, 5.29833973, 8.01541...             2   
1       171  [2.030374637, 4.634829893, 6.940987113, 8.5359...             2   
2       172  [3.992375845, 8.080901131, 12.34340583, 16.059...             1   
3       173  [4.159753575, 7.654566962, 11.77586291, 16.585...             1   
4       174  [2.140333354, 3.668254357, 6.18842916, 8.16284...             2   
5       175  [4.312450305, 8.084623177, 11.48604125, 15.186...             1   
6       176  [4.57722904, 8.046616572, 11.29743912, 15.8076...             1   
7       177  [3.335158198, 8.351921916, 12.63921605, 15.635...             1   
8       178  [2.068326985, 4.647149543, 6.334827587, 8.0155...             2   

   predicted_class  
0          